In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import utils.text_processing as tp

In [7]:
pmethod = 'typos'

In [4]:
%store -r important_words
%store -r sentence_packages
%store -r ds
%store -r advds
%store -r extended_original_predictions
%store -r extended_modified_predictions

In [25]:
len(extended_modified_predictions)

2092

In [30]:
def generate_results_df(pmethod, ds, advds, extended):  
    
    if       
    results = pd.DataFrame({
     'Perturbation Method': [pmethod],
     'Tokenizer': ['nlptown/bert-base-multilingual-uncased-sentiment'], 
     'Model' : ['nlptown/bert-base-multilingual-uncased-sentiment'], 
     'Dataset':['TripAdvisor Hotel Reviews'], 
     'Output lables': ['Range from 0 to 4 - 0 = NEG; 4 = POS'],
     'Items in original dataset': len(ds),
     'Number of modifyable items in original dataset': len(advds),
     'Items in adversarial dataset': len(extended),
     'Percentage': (len(advds)/len(ds)*100)})
    
    return results.T

In [31]:
results_typo = generate_results_df(pmethod, ds, advds, extended_modified_predictions)
results_typo

,0
Perturbation Method,typos
Tokenizer,nlptown/bert-base-multilingual-uncased-sentiment
Model,nlptown/bert-base-multilingual-uncased-sentiment
Dataset,TripAdvisor Hotel Reviews
Output lables,Range from 0 to 4 - 0 = NEG; 4 = POS
Items in original dataset,435
Number of modifyable items in original dataset,330
Items in adversarial dataset,2092
Percentage,75.8621


### Dataloader

In [ ]:
def semeval_to_aspectsentiment_hr(filename, with_ids=False):
    sentimap = {
        'positive': 'POS',
        'negative': 'NEG',
        'neutral': 'NEU'
    }

    def transform_category_name(s):
        return s

    with open(filename) as file:

        review_elements = ET.parse(file).getroot().iter('Review')

        sentences = []
        aspect_category_sentiments = []
        classes = set([])
        sentence_ids = []

        for j, review_element in enumerate(review_elements):
            # review_text = ' '.join([el.text for el in review_element.iter('text')])

            for i, s in enumerate(review_element.iter('sentence')):
                s_id = s.get('id')
                sentence_ids.append(s_id)
                sentence_text = s.find('text').text
                aspect_category_sentiment = set([])
                for o in s.iter('Opinion'):
                    aspect_category = transform_category_name(o.get('category'))
                    classes.add(aspect_category)
                    sentiment = sentimap[o.get('polarity')]
                    aspect_category_sentiment.add((aspect_category, sentiment))

                
                aspect_sentiment_dict = {}
                for asentis in aspect_category_sentiment:
                    aspect_sentiment_dict[asentis[0]] = asentis[1]
                    sentences.append(sentence_text)
                    aspect_category_sentiments.append(aspect_sentiment_dict)

        cats = list(classes)
        cats.sort()

    idx2aspectlabel = {k: v for k, v in enumerate(cats)}
    sentilabel2idx = {"NONE": 0, "NEG": 1, "NEU": 2, "POS": 3}
    idx2sentilabel = {k: v for v, k in sentilabel2idx.items()}
    if not with_ids:
        return sentences, aspect_category_sentiments, (idx2aspectlabel, idx2sentilabel)
    else:
        return sentences, sentence_ids, aspect_category_sentiments, (idx2aspectlabel, idx2sentilabel)

